In [16]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn

from scipy.io import loadmat
from torch.autograd import Variable
from torchvision.models import efficientnet_b0
from torch.nn import (
    AdaptiveAvgPool2d,
    Conv2d,
    Linear,
    SiLU,
    Sigmoid
)

from numpy_utils.layers import (
    Layer,
    LinearLayer,
    ConvLayer,
    SiLULayer,
    SigmoidLayer,
    CrossEntropyLoss,
    CrossEntropyCost,
    SoftmaxLayer
)
from data_utils.dataset import CarsDataset

In [2]:
# plt.rcParams['figure.figsize'] = (15, 5)
%load_ext autoreload
%autoreload 2

In [5]:
def annot_mat_to_csv(mat_path, csv_path):
    meta = loadmat(mat_path)
    annots = meta['annotations']
    c_name = annots[:, :][['class', 'fname']].squeeze(0)
    c = list(map(lambda x: x.item(), c_name['class']))
    name = list(map(lambda x: x.item(), c_name['fname']))
    df = pd.DataFrame({
        'class': c,
        'fname': name
    })
    df.to_csv(csv_path, index=False)

In [6]:
annot_mat_to_csv('../data/cars_train_annos.mat', '../data/train.csv')

In [112]:
dts = CarsDataset('../data/train', '../data/train.csv')

In [132]:
it = iter(dts)
hws = np.array((list(map(lambda x: x[0].size, it))))
h, w = map(lambda x: x.squeeze(1), np.split(hws, 2, axis=1))
print(h.shape, w.shape)

(8144,) (8144,)


In [135]:
print(len(np.unique(h)))
print(len(np.unique(w)))

654
829


In [31]:
def compare(X, ft, fn, ac=True):
    out = ft(X).detach().numpy()
    outn = fn(X.detach().numpy())
    print('MSE', np.power(out - outn, 2).sum())
    if ac:
        print('eq', np.allclose(out, outn))

In [19]:
model = efficientnet_b0(weights=None)

In [7]:
lt = Linear(512, 128)
ln = LinearLayer(512, 128)

In [12]:
ln.W = lt.weight.data.numpy()
ln.b = lt.bias.data.numpy()

In [13]:
X = torch.rand((1, 512))
Xn = X.numpy()

In [14]:
out = lt(X).detach().numpy()
outn = ln(Xn)
print(np.power(out - outn, 2).sum())
print(np.allclose(out, outn))

6.5755734e-13
False


In [15]:
ct = Conv2d(3, 4, 3)
cn = ConvLayer(3, 4, 3)
wt, bt = ct._parameters.values()
# cn.W = np.transpose(wt.data.numpy(), (0, 1, 2, 3))
cn.W = wt.data.numpy()
cn.b = bt.data.numpy()

In [16]:
X = torch.rand(4, 3, 5, 5)
Xn = X.numpy()
# Xn = np.arange(48).reshape(1, 3, 4, 4)

In [17]:
out = ct(X).detach().numpy()
outn = cn(Xn)
print(*(x.shape for x in (out, outn)))

(4, 4, 3, 3) (4, 4, 3, 3)


In [18]:
print(np.power(out - outn, 2).sum())
# print(np.power(out - outn2, 2).sum())
# print(np.power(out - outn3, 2).sum())
# print(np.power(out - outn4, 2).sum())
print(np.allclose(out, outn))
# print(np.allclose(out, outn2))
# print(np.allclose(out, outn3))
# print(np.allclose(out, outn4))

2.0724481e-13
True


In [41]:
X = torch.rand(5, 3, 4, 4)
compare(X, SiLU(), SiLULayer())
compare(X, Sigmoid(), SigmoidLayer())

MSE 1.339315e-13
eq True
MSE 1.314504e-13
eq True


In [44]:
X = torch.rand(5, 3, 16, 16)
avgpool = AdaptiveAvgPool2d(output_size=1)
out = avgpool(X)
print(out.shape)

torch.Size([5, 3, 1, 1])


In [20]:
class Md(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.l1 = nn.Linear(8, 4)
        self.act1 = nn.Sigmoid()

    def forward(self, X):
        x = self.l1(X)
        x = self.act1(x)
        return x

In [79]:
# md = Md()
md = nn.Linear(8, 4)
mdn = LinearLayer(8, 4)
mdn.W = md.weight.data.numpy()
mdn.b = md.bias.data.numpy()
crit = nn.CrossEntropyLoss(reduction='mean')
critn = CrossEntropy()

In [80]:
X = torch.rand(5, 8)
lab = torch.randint(0, 4, (5,))
Xn = X.numpy()
labn = lab.numpy()

In [82]:
out = md(X)
outn = mdn(Xn)
loss = crit(out, lab)
lossn = critn(outn, labn)

tensor(1.3982, grad_fn=<NllLossBackward0>)
1.3982131
True


In [17]:
crit = nn.CrossEntropyLoss(reduction='mean')
critn = CrossEntropyLoss()
sm = SoftmaxLayer()
cre = CrossEntropyCost()

In [20]:
X = torch.rand(5, 4, requires_grad=True)
lab = torch.randint(0, 4, (5,))
Xn = X.detach().numpy()
labn = lab.numpy()

In [21]:
out1 = crit(X, lab)
out2 = critn(Xn, labn)
_out3 = sm(Xn)
out3 = cre(_out3, labn)
print(np.allclose(out1.detach(), out3))
print(np.allclose(out1.detach(), out2))

True
True


In [22]:
out1.backward()
print(X.grad)
print(X)

tensor([[ 0.0771,  0.0488,  0.0320, -0.1580],
        [ 0.0492,  0.0373,  0.0762, -0.1627],
        [ 0.0536,  0.0608,  0.0335, -0.1479],
        [ 0.0851, -0.1609,  0.0413,  0.0345],
        [ 0.0681, -0.1425,  0.0428,  0.0316]])
tensor([[0.9990, 0.5419, 0.1202, 0.3906],
        [0.4233, 0.1456, 0.8611, 0.1475],
        [0.6178, 0.7426, 0.1484, 0.5884],
        [0.9221, 0.1434, 0.2000, 0.0206],
        [0.9788, 0.8103, 0.5140, 0.2102]], requires_grad=True)


In [10]:
back = cre.backward()
print(back)
back1 = sm.backward(back)
print(back1)

[[-3.7234988  0.         0.         0.       ]
 [ 0.         0.        -3.5413024  0.       ]
 [ 0.        -4.2947288  0.         0.       ]
 [ 0.        -3.8107617  0.         0.       ]
 [-3.7204578  0.         0.         0.       ]]
[[-0.7314354   0.15230446  0.21740596  0.36172494]
 [ 0.29056123  0.14554171 -0.71761805  0.2815151 ]
 [ 0.20540085 -0.7671564   0.21833926  0.34341627]
 [ 0.29781467 -0.73758525  0.16264066  0.27713   ]
 [-0.7312159   0.17046641  0.26819462  0.29255483]]
